In [11]:
#enter file name
file = 'westnile_dataset.csv'
#take out DC depending on map

In [16]:
import pandas as pd
import numpy as np

df = pd.read_csv(file)
df = df.loc[df['PartOfCumulativeCountSeries'] == 1]
df = df[['Admin1Name', 'PeriodStartDate', 'PeriodEndDate', 'CountValue']]
df = df.rename(columns={'Admin1Name':'State',
                        'PeriodStartDate':'Start',
                        'PeriodEndDate':'End',
                        'CountValue':'Count'})
df = df.loc[df['Start'] > '2009-12-31']
df.sort_values(by=['State', 'End', 'Start'], inplace=True)
df.reset_index(inplace=True, drop=True)

states = ['ALABAMA', 'ALASKA', 'ARIZONA', 'ARKANSAS', 'CALIFORNIA', 'COLORADO', 'CONNECTICUT', 'DELAWARE', 'DISTRICT OF COLUMBIA', 'FLORIDA', 'GEORGIA', 'HAWAII', 'IDAHO', 'ILLINOIS', 'INDIANA', 'IOWA', 'KANSAS', 'KENTUCKY', 'LOUISIANA', 'MAINE', 'MARYLAND', 'MASSACHUSETTS', 'MICHIGAN', 'MINNESOTA', 'MISSISSIPPI', 'MISSOURI', 'MONTANA', 'NEBRASKA', 'NEVADA', 'NEW HAMPSHIRE', 'NEW JERSEY', 'NEW MEXICO', 'NEW YORK', 'NORTH CAROLINA', 'NORTH DAKOTA', 'OHIO', 'OKLAHOMA', 'OREGON', 'PENNSYLVANIA', 'RHODE ISLAND', 'SOUTH CAROLINA', 'SOUTH DAKOTA', 'TENNESSEE', 'TEXAS', 'UTAH', 'VERMONT', 'VIRGINIA', 'WASHINGTON', 'WEST VIRGINIA', 'WISCONSIN', 'WYOMING']
states_df = pd.DataFrame(states)
states_df.rename(columns={0:'State'}, inplace=True)
template_df = pd.DataFrame(states)
template_df.rename(columns={0:'State'}, inplace=True)

yearly = pd.read_csv('yearlycensus.csv')
USpop = yearly.loc[0,'CENSUS2010POP']

finalyear = df['Start'].iloc[-1]
finalyear = int(finalyear[0:4])
timerange = np.arange(2010, finalyear+1)

for year in timerange:
    year_df = [[]]
    year_df = df.loc[(df.Start.str.startswith(str(year)))].append(pd.Series(index=()), ignore_index=True)
    year_data = []
    lastrow = len(year_df) - 1

    for index, column in year_df.iterrows():
        if (index < lastrow) and (year_df.iloc[index,0] != year_df.iloc[(index+1),0]):
            year_data.append(year_df.iloc[index])

    cleaned_year_df = pd.DataFrame()
    cleaned_year_df = pd.DataFrame(year_data)
    cleaned_year_df.reset_index(inplace=True, drop=True)
    cleaned_year_df.drop(columns=['Start', 'End'], inplace=True)
    cleaned_year_df = cleaned_year_df.rename(columns={'Count':'Cases_'+str(year)})

    template_df = template_df.merge(cleaned_year_df, on='State', how='outer')
    merged_df = states_df.merge(template_df, on='State', how='left')
    
for year in timerange:
    merged_df['Rate_'+str(year)] = merged_df['Cases_'+str(year)] / USpop * 100000
    
merged_df.set_index('State', inplace=True, drop=True)
merged_df = merged_df.fillna(0)
merged_df['TotalByState'] = merged_df.sum(axis=1)
merged_df.loc['TotalByYear'] = merged_df.sum()

merged_df.to_csv('cleaned_' + file)

In [17]:
merged_df

,Cases_2010,Cases_2011,Cases_2012,Cases_2013,Cases_2014,Cases_2015,Cases_2016,Cases_2017,Rate_2010,Rate_2011,Rate_2012,Rate_2013,Rate_2014,Rate_2015,Rate_2016,Rate_2017,TotalByState
State,,,,,,,,,,,,,,,,,
ALABAMA,2.0,0.0,7.0,0.0,2.0,5.0,9.0,26.0,0.000648,0.000000,0.002267,0.000000,0.000648,0.001619,0.002915,0.008421,51.016518
ALASKA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ARIZONA,58.0,20.0,10.0,2.0,24.0,35.0,19.0,18.0,0.018786,0.006478,0.003239,0.000648,0.007773,0.011336,0.006154,0.005830,186.060244
ARKANSAS,1.0,0.0,2.0,0.0,2.0,2.0,3.0,3.0,0.000324,0.000000,0.000648,0.000000,0.000648,0.000648,0.000972,0.000972,13.004211
CALIFORNIA,38.0,45.0,135.0,6.0,251.0,186.0,89.0,133.0,0.012308,0.014575,0.043725,0.001943,0.081297,0.060244,0.028826,0.043078,883.285996
COLORADO,55.0,5.0,228.0,5.0,69.0,44.0,90.0,39.0,0.017814,0.001619,0.073847,0.001619,0.022349,0.014251,0.029150,0.012632,535.173282
CONNECTICUT,1.0,1.0,3.0,0.0,3.0,2.0,0.0,1.0,0.000324,0.000324,0.000972,0.000000,0.000972,0.000648,0.000000,0.000324,11.003563
DELAWARE,2.0,0.0,7.0,0.0,0.0,6.0,0.0,1.0,0.000648,0.000000,0.002267,0.000000,0.000000,0.001943,0.000000,0.000324,16.005182
DISTRICT OF COLUMBIA,1.0,10.0,2.0,1.0,2.0,1.0,0.0,3.0,0.000324,0.003239,0.000648,0.000324,0.000648,0.000324,0.000000,0.000972,20.006478
